# Horses or Humans Classification

---
```
Step 0. Library Import
Step 1. Dataset Load
Step 2. Data Preprocessing
Step 3. Modeling
Step 4. Model Compile
Step 5. Model Checkpoint
Step 6. Model Fit
Step 7. Model Evaluate & Svae
Step 8. Reload Model
```

## Step 0. Library Import

In [1]:
import urllib.request
import zipfile
import numpy as np
from IPython.display import Image

import tensorflow as tf 
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dropout, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint

## Step 1. Dataset Load

In [2]:
train_url = "https://storage.googleapis.com/download.tensorflow.org/data/horse-or-human.zip"
valid_url = "https://storage.googleapis.com/download.tensorflow.org/data/validation-horse-or-human.zip"

In [3]:
train_zipfile_path = './data/train_horses_or_humans.zip'
valid_zipfile_path = './data/valid_horses_or_hamans.zip'

In [13]:
# train set unzip
urllib.request.urlretrieve(train_url, train_zipfile_path)
zip_ref = zipfile.ZipFile(train_zipfile_path,'r')
zip_ref.extractall('./data/train_horses_or_humans/')
zip_ref.close()

# test set unzip
urllib.request.urlretrieve(valid_url, valid_zipfile_path)
zip_ref2 = zipfile.ZipFile(valid_zipfile_path,'r')
zip_ref2.extractall('./data/valid_horses_or_humans/')
zip_ref2.close()

## Step 2. Data Preprocessing

In [14]:
train_data_path = './data/train_horses_or_humans/'
valid_data_path = './data/valid_horses_or_humans/'

In [34]:
train_idg = ImageDataGenerator(
    rescale = 1 / 255.0,
    rotation_range = 5,
    width_shift_range = 0.05,
    height_shift_range = 0.05,
    zoom_range = 0.3,
    horizontal_flip = True,
    fill_mode = 'nearest',
)

In [35]:
valid_idg = ImageDataGenerator(
    rescale = 1 / 255.0,
)

In [36]:
train_gen = train_idg.flow_from_directory(
    train_data_path,
    target_size=(300,300),
    batch_size=64,
    class_mode='binary'
)

Found 1027 images belonging to 2 classes.


In [37]:
valid_gen = valid_idg.flow_from_directory(
    valid_data_path,
    target_size=(300,300),
    batch_size=64,
    class_mode='binary'
)

Found 256 images belonging to 2 classes.


## Step 3. Modeling

In [38]:
model = Sequential([
    Conv2D(16, (3, 3), activation='relu', input_shape=(300, 300, 3)),
    MaxPooling2D(2, 2),
    Conv2D(32, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')
])

In [39]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_15 (Conv2D)           (None, 298, 298, 16)      448       
_________________________________________________________________
max_pooling2d_15 (MaxPooling (None, 149, 149, 16)      0         
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 147, 147, 32)      4640      
_________________________________________________________________
max_pooling2d_16 (MaxPooling (None, 73, 73, 32)        0         
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 71, 71, 64)        18496     
_________________________________________________________________
max_pooling2d_17 (MaxPooling (None, 35, 35, 64)        0         
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 33, 33, 64)       

## Step 4. Model Compile

In [40]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

## Step 5. Model Checkpoint

In [41]:
filepath = 'my_checkpoint.ckpt'
cp = ModelCheckpoint(
    filepath=filepath,
    save_weights_only=True,
    save_best_only=True,
    monitor='val_loss',
    verbose=1
)

## Step 6. Model Fit

In [42]:
model.fit(
    train_gen,
    validation_data=(valid_gen),
    epochs=12,
    callbacks=[cp]
)

Epoch 1/12
17/17 [==============================] - 27s 2s/step - loss: 0.6814 - acc: 0.5107 - val_loss: 1.3889 - val_acc: 0.5352

Epoch 00001: val_loss improved from inf to 1.38892, saving model to my_checkpoint.ckpt
Epoch 2/12
17/17 [==============================] - 26s 2s/step - loss: 0.3820 - acc: 0.8077 - val_loss: 1.0701 - val_acc: 0.8242

Epoch 00002: val_loss improved from 1.38892 to 1.07015, saving model to my_checkpoint.ckpt
Epoch 3/12
17/17 [==============================] - 26s 2s/step - loss: 0.2193 - acc: 0.9116 - val_loss: 1.5757 - val_acc: 0.7344

Epoch 00003: val_loss did not improve from 1.07015
Epoch 4/12
17/17 [==============================] - 26s 2s/step - loss: 0.1396 - acc: 0.9524 - val_loss: 1.1500 - val_acc: 0.7656

Epoch 00004: val_loss did not improve from 1.07015
Epoch 5/12
17/17 [==============================] - 26s 2s/step - loss: 0.2034 - acc: 0.9286 - val_loss: 2.0972 - val_acc: 0.7812

Epoch 00005: val_loss did not improve from 1.07015
Epoch 6/12
17/

## Step 7. Model Evaluate & Save

In [43]:
model.load_weights(filepath)

In [44]:
model.evaluate(valid_gen)

4/4 [==============================] - 1s 256ms/step - loss: 0.7197 - acc: 0.8125


[0.7196962237358093, 0.8125]

In [45]:
model.save('./model/hoh_cnn_7197.h5')

## Step 8. Reload Model

In [48]:
mymodel = tf.keras.models.load_model('./model/hoh_cnn_7197.h5')

In [49]:
mymodel.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_15 (Conv2D)           (None, 298, 298, 16)      448       
_________________________________________________________________
max_pooling2d_15 (MaxPooling (None, 149, 149, 16)      0         
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 147, 147, 32)      4640      
_________________________________________________________________
max_pooling2d_16 (MaxPooling (None, 73, 73, 32)        0         
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 71, 71, 64)        18496     
_________________________________________________________________
max_pooling2d_17 (MaxPooling (None, 35, 35, 64)        0         
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 33, 33, 64)       